In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# выбрать device на котором запускать
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# определение гиперпараметров 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [2]:
import random
import numpy

def set_seed(seed: int, n_gpu: int):
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)
        
set_seed(42, 1)

In [3]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root="../data", 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root="../data", 
                                          train=False, 
                                          transform=transforms.ToTensor())

# загрузка данных
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw



Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw


Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw
Processing...


C:\Users\Lost\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:439: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [4]:
# Определение архитектуры нейронной сети
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [5]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model.train()

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [6]:

# loss и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# тренировка модели
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # перенос тензоров на определенный device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0123
Epoch [1/5], Step [200/600], Loss: 0.0062
Epoch [1/5], Step [300/600], Loss: 0.0141
Epoch [1/5], Step [400/600], Loss: 0.0072
Epoch [1/5], Step [500/600], Loss: 0.0479
Epoch [1/5], Step [600/600], Loss: 0.0680
Epoch [2/5], Step [100/600], Loss: 0.0255
Epoch [2/5], Step [200/600], Loss: 0.0182
Epoch [2/5], Step [300/600], Loss: 0.0121
Epoch [2/5], Step [400/600], Loss: 0.0138
Epoch [2/5], Step [500/600], Loss: 0.0203
Epoch [2/5], Step [600/600], Loss: 0.0581
Epoch [3/5], Step [100/600], Loss: 0.0052
Epoch [3/5], Step [200/600], Loss: 0.0190
Epoch [3/5], Step [300/600], Loss: 0.0067
Epoch [3/5], Step [400/600], Loss: 0.0027
Epoch [3/5], Step [500/600], Loss: 0.0094
Epoch [3/5], Step [600/600], Loss: 0.0386
Epoch [4/5], Step [100/600], Loss: 0.0030
Epoch [4/5], Step [200/600], Loss: 0.0047
Epoch [4/5], Step [300/600], Loss: 0.0078
Epoch [4/5], Step [400/600], Loss: 0.0518
Epoch [4/5], Step [500/600], Loss: 0.0050
Epoch [4/5], Step [600/600], Loss:

In [9]:
model.eval()

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)